In [1]:
MODEL_NAME = "models/gru_att_01"

In [2]:
import nltk
import numpy
import pickle
numpy.random.seed(42)

from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter


from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json, load_model


Using TensorFlow backend.


In [4]:
max_features = 300000
maxlen = 300

In [17]:
import re

from nltk.tokenize import word_tokenize, TweetTokenizer 

tweet_tokenizer = TweetTokenizer()
TOKEN_LINK = "URL"
PUNCTUATION = {'.', '?', '!', ';', ''}

def replace_links(text, token=TOKEN_LINK):
    # some links begin with http or https: https?://\S+
    # link can begins with www: www\.\S+
    # links to google documents: \\bgoo\.gl/\w+/?
    # else:
    #   hostname usually have some domain names separated by dot: (\w+\.){2,4}
    #   last domain name can have min 2 and max 6 letters: ([a-z]{2,6})
    #   if path exists - it can be separated by / and must begins with /: (/\S+)*
    #   link can have / in the end: /?
    # to match some links with 1 dot we check if exists path after hostname:
    #   hostname: \w+\.([a-z]{2,6})
    #   path: /(\S+/)*(\S+)?
    token = ' ' + token + ' '
    return re.sub("https?://\S+|www\.\S+|\\bgoo\.gl/\w+/?|"
                  "(\w+\.){2,4}([a-z]{2,6})(/\S+)*/?|\w+\.([a-z]{2,6})/(\S+/)*(\S+)?", token, text)


def tokenize(text, link_token=TOKEN_LINK):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
#     text = re.sub(' u ', 'you', text)
#     text = re.sub('\nu ', 'you', text)
#     text = re.sub(' u\n', 'you', text)    
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub("fuck", ' fuck ', text)
    text = re.sub("idiot", ' idiot ', text)
    text = re.sub("stupid", ' stupid ', text)
    text = re.sub("dick", ' dick ', text)
    text = re.sub("shit", ' shit ', text)
    text = text.strip(' ')
#     return text
    
    text = re.sub(r"_", " ", text)
    text = re.sub(r"-", " ", text)

    text = replace_links(text, link_token)

    tokens = [t for t in tweet_tokenizer.tokenize(text)]
    # Remove hashtags
    tokens=[re.sub(r'#', '', i) for i in tokens]
    tokens=['.' if i == '..' else i for i in tokens]
    return ' '.join(tokens)

In [6]:
#load tokenizer
pkl_file = open(MODEL_NAME+'_tokenizer', 'rb')
tok = pickle.load(pkl_file)
pkl_file.close()

In [8]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, CuDNNLSTM
from keras.layers.merge import concatenate
from keras.models import Model, Layer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import initializers, regularizers, constraints
import keras.backend as K
from keras.layers import Conv1D, GaussianNoise

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        super(AttentionWithContext, self).build(input_shape)
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
#         weighted_input = x * a
#         return K.sum(weighted_input, axis=1)
        return a
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [10]:
model = load_model(MODEL_NAME + '.h5', {'AttentionWithContext': AttentionWithContext}, )

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [11]:
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function

In [12]:
index_to_word = {index: word for word, index in tok.word_index.items()}

In [13]:
def preprocess(text):
    text = tokenize(text)
    return sequence.pad_sequences(tok.texts_to_sequences([text]), maxlen=maxlen)

In [14]:
def get_attention(text):
    rez = []
    word_indexes = preprocess(text)
    layer_outs = functor([word_indexes, 0.])
    attention_weights = [w[0] for w in layer_outs[-7][0]]
    for word_index, attention in zip(word_indexes[0], attention_weights):
#         if word_index != 0:
            rez.append((index_to_word.get(word_index, 0), attention))
    return rez

In [15]:
def get_summary(attention, percentile=95):
    summary = []
    threshold = np.percentile([a[1]for a in attention], percentile)
    for word, weight in attention:
        if weight > threshold:
            summary.append(word)
    return ' '.join(summary)

In [18]:
text = """
*My niggeez Auranor know 'sup. Neea know how 2 agree wit a muthafucka, niggaz. 
"""
attention = get_attention(text)
print(tokenize(text))
print('Predicttion:', model.predict(preprocess(text)))
# get_summary(attention)
sorted(attention, key=lambda x: x[1], reverse=True)

my niggeez auranor know up neea know how 2 agree wit a mutha fuck a niggaz
Predicttion: [[0.98616725 0.21777329 0.97447383 0.02286493 0.91591907 0.925846  ]]


[('fuck', 0.4432374),
 ('mutha', 0.2418755),
 ('niggaz', 0.17667122),
 ('a', 0.1127302),
 ('wit', 0.013007024),
 ('a', 0.011463671),
 ('agree', 0.00039864704),
 ('up', 0.00028695018),
 ('2', 0.00021746528),
 ('how', 5.9453516e-05),
 ('know', 2.7019723e-05),
 ('my', 1.7767823e-05),
 ('know', 7.707409e-06),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),
 (0, 0.0),


In [ ]:
tokenize()